<a href="https://colab.research.google.com/github/ishan-anuranga/PySparkConcepts/blob/main/PySpark_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=5f0272846b1ab5d3f8cabd204ed723de4158ecf197c4cd2ffc92962aae012986
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext('local', 'PySparkIntro')

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('PySparkIntro').getOrCreate()